## Starter Code

<span style="color:blue">**In this guide, we provide a starter code to help you begin your project. 
Please feel free to use it thoughtfully and tailor it to your specific requirements.**</span>

### Dataset

**Facial Recognition**

Dataset: FER2013
- CSV Format: Kaggle Link https://www.kaggle.com/datasets/nicolejyt/facialexpressionrecognition
- JPG Format: Kaggle Link https://www.kaggle.com/datasets/msambare/fer2013/data
- Details:
    - 34,034 images
    - 48x48 pixels
    - 7 emotions: Angry, Disgust, Fear, Happy, Sad, Surprise, Neutral

**Gesture Recognition**

Dataset: LeapGestureRecognition

- Download Link: Kaggle https://www.kaggle.com/datasets/gti-upm/leapgestrecog
- Other Gesture Dataset: GitHub https://github.com/linto-ai/multi-hand-gesture-dataset

### Examples & Inspirations:

#### Facial Expression

1. Static and dynamic facial emotion recognition using the Emo-AffectNet 

   https://huggingface.co/ElenaRyumina/face_emotion_recognition


   ![example](https://github.com/ElenaRyumina/EMO-AffectNetModel/blob/main/gif/result_2.gif?raw=true)


2. Facial Expression Recognition Challenge

   https://github.com/chinhau-lim/fer_2013

#### Hand Gesture

1. Real-time hand gesture recognition using TensorFlow & OpenCV

   https://techvidvan.com/tutorials/hand-gesture-recognition-tensorflow-opencv/

    ![example](https://techvidvan.com/tutorials/wp-content/uploads/sites/2/2021/07/landmark-output.gif)



2. Deep_learning_hand_gesture_recognition

   https://github.com/guillaumephd/deep_learning_hand_gesture_recognition

In [1]:
# Install required packages
# !pip3 install opencv-python numpy pandas tensorflow scikit-learn

In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight


ModuleNotFoundError: No module named 'cv2'

### 1. Data Preparation

In [ ]:
# Facial Emotion Classes
EMOTION_MAP = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Gesture Classes (LeapGestRecog)
GESTURE_MAP = {
    0: 'palm',        # 01_palm
    1: 'l',           # 02_l
    2: 'fist',        # 03_fist
    3: 'fist_moved',  # 04_fist_moved
    4: 'thumb',       # 05_thumb
    5: 'index',       # 06_index
    6: 'ok',          # 07_ok
    7: 'palm_moved',  # 08_palm_moved
    8: 'c',           # 09_c
    9: 'down'         # 10_down
}

def load_fer2013(file_path='fer2013.csv'):
    """Loads and balances FER2013 dataset"""
    df = pd.read_csv(file_path)
    pixels = df['pixels'].apply(lambda x: np.array(x.split(), dtype='float32'))
    images = np.array([x.reshape(48, 48, 1) for x in pixels]) / 255.0
    labels = df['emotion'].values
    
    # Handle class imbalance
    class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
    return images, labels, dict(enumerate(class_weights))

def load_leap_gestures(dataset_path="LeapGestRecog"):
    """Loads Leap Motion dataset with correct label mapping"""
    X, y = [], []
    
    for root, _, files in os.walk(dataset_path):
        try:
            folder_name = os.path.basename(root)
            if not folder_name[:2].isdigit():
                continue
                
            gesture_num = int(folder_name[:2]) - 1
            if gesture_num not in GESTURE_MAP:
                continue
                
            # Process images
            for file in files:
                if file.endswith(".png"):
                    img = cv2.imread(os.path.join(root, file), cv2.IMREAD_GRAYSCALE)
                    img = cv2.resize(img, (64, 64))
                    X.append(img.astype('float32') / 255.0)
                    y.append(gesture_num)
                    
        except Exception as e:
            print(f"Skipping {root}: {str(e)}")
    
    return np.array(X).reshape(-1, 64, 64, 1), np.array(y)

### 2. Model Architecture

In [ ]:
class EmotiveAvatarSystem:
    def __init__(self):
        self.face_model = self.build_face_model()
        self.gesture_model = self.build_gesture_model()
        
    def build_face_model(self):
        model = models.Sequential([
            layers.Conv2D(32, (3,3), activation='relu', input_shape=(48,48,1)),
            layers.MaxPooling2D((2,2)),
            layers.Dropout(0.3),
            
            layers.Conv2D(64, (3,3), activation='relu'),
            layers.MaxPooling2D((2,2)),
            layers.Dropout(0.4),
            
            layers.GlobalAveragePooling2D(),
            layers.Dense(128, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(7, activation='softmax')
        ])
        
        model.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
        return model
    
    
    def build_gesture_model(self):
        model = models.Sequential([
            layers.Conv2D(16, (3,3), activation='relu', input_shape=(64,64,1)),
            layers.MaxPooling2D((2,2)),
            layers.Dropout(0.25),
            
            layers.Conv2D(32, (3,3), activation='relu'),
            layers.MaxPooling2D((2,2)),
            layers.Dropout(0.35),
            
            layers.GlobalAveragePooling2D(),
            layers.Dense(128, activation='relu'),
            layers.Dropout(0.5),
            layers.Dense(10, activation='softmax')
        ])
        
        model.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
        return model

### 3. Training & Evaluation

In [ ]:
if __name__ == "__main__":
    X_face, y_face, face_weights = load_fer2013()
    X_gesture, y_gesture = load_leap_gestures()
    
    # Split datasets
    Xf_train, Xf_test, yf_train, yf_test = train_test_split(X_face, y_face, test_size=0.2, stratify=y_face)
    Xg_train, Xg_test, yg_train, yg_test = train_test_split(X_gesture, y_gesture, test_size=0.2, stratify=y_gesture)
    
    # Initialize system
    avatar = EmotiveAvatarSystem()
    
    # Train facial model
    print("Training Facial Model (FER2013)...")
    avatar.face_model.fit(Xf_train, yf_train,
                         epochs=5,
                         batch_size=64,
                         class_weight=face_weights,
                         validation_split=0.2,
                         callbacks=[
                             callbacks.EarlyStopping(patience=5, restore_best_weights=True),
                             callbacks.ReduceLROnPlateau(factor=0.5, patience=2)
                         ])
    
    # Train gesture model
    print("\nTraining Gesture Model (Leap Motion)...")
    avatar.gesture_model.fit(Xg_train, yg_train,
                            epochs=5,
                            batch_size=32,
                            validation_split=0.2,
                            callbacks=[
                                callbacks.EarlyStopping(patience=3)
                            ])
    
    # Evaluate
    f_loss, f_acc = avatar.face_model.evaluate(Xf_test, yf_test)
    g_loss, g_acc = avatar.gesture_model.evaluate(Xg_test, yg_test)
    
    print("\n=== Final Performance ===")
    print(f"Facial Accuracy: {f_acc:.2%}")
    print(f"Gesture Accuracy: {g_acc:.2%}")

/Users/duan.425/Library/Python/3.10/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training Facial Model (FER2013)...
Epoch 1/5
359/359 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.1265 - loss: 1.9515 - val_accuracy: 0.1855 - val_loss: 1.9190 - learning_rate: 0.0010
Epoch 2/5
359/359 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.1673 - loss: 1.9090 - val_accuracy: 0.1804 - val_loss: 1.9212 - learning_rate: 0.0010
Epoch 3/5
359/359 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.1765 - loss: 1.9311 - val_accuracy: 0.1942 - val_loss: 1.8988 - learning_rate: 0.0010
Epoch 4/5
359/359 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.2010 - loss: 1.8664 - val_accuracy: 0.1296 - val_loss: 1.9502 - learning_rate: 0.0010
Epoch 5/5
359/359 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 0.1636 - loss: 1.8952 - val_accuracy: 0.1827 - val_loss: 1.9028 - learning_rate: 0.0010

Training Gesture Model (Leap Motion)...
Epoch 1/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.1105 - loss: 2.3009 - val_accuracy: 0.0938 - val_loss: 2.2951
Epoch 2/5
40/40 ━━━━━━━━━━━━━━━━━━━━ 0

### Improvement Opportunities:
1. Add real-time webcam integration
2. Implement data augmentation for infrared images
4. Add attention mechanisms to the CNN
5. ......

Reference:

https://github.com/chinhau-lim/fer_2013 

https://arxiv.org/pdf/2105.03588

https://github.com/takanto/FER2013

https://aicompetence.org/ai-in-cultural-sensitivity-gesture-interpretation/
